In [15]:
from General_Raster_Funcs.RasterTiling import getTiles
from General_Raster_Funcs.TiffManagement import *

In [16]:
from raster_aggregation.temporal_aggregation import TemporalAggregator_Dynamic

In [17]:
import glob
import numpy as np
from osgeo import gdal
import os
from collections import defaultdict

In [18]:
inFilePattern = r'C:\Temp\tsmodel\1km\local\martens2_1k_global_fixhr\out_combined\*.tif'
what = "TSI-Martens2-Pf"
res = "1km"

In [29]:
outDir = r"C:\Temp\tsmodel\1km\local\martens2_1k_global_fixhr\annual_combined"
tileDir = r"C:\Temp\tsmodel\1km\local\martens2_1k_global_fixhr\annual_combined\tiles"

In [21]:
# Specify the height of each tile - depends on available memory.
# The algorithm will work in slices that are this height and full-width,
# so the algorithm needs around idealSlice * fullWidth * 80 bytes of RAM
# Thus with global 1k images (43200px wide), a slice of 7168 (high) needs 
# around 25Gb RAM. Choose based on your PC's RAM.

# The rasters have tilesize 256 (or a multiple thereof) so pick a size
# that is a multiple of this where possible for most efficient access
# (not a big deal, though)
idealSlice = 7200
idealSlice = 8700
# alter to suit the images
fullWidth = 43200
fullHeight = 17400
#fullWidth = 8640
#fullHeight = 3480

# alter output no data value to whatever you want: not required to be the same as the inputs
outNdv = -9999


In [22]:
allfiles = glob.glob(inFilePattern)

In [23]:
allFilesDict = defaultdict(list)
for fn in allfiles:
    parts = os.path.basename(fn).split('.')
    yr = int(parts[1])
    allFilesDict[str(yr)].append(fn)

In [24]:
stats = ["mean", "min", "max", "sd"]

In [25]:
slices = sorted(list(set([s[1] for s in getTiles(fullWidth, fullHeight, idealSlice)])))

In [27]:
slices

[(0, 8700), (8700, 17400)]

In [28]:
def fnGetter(what, when, stat, where):
    if where == -1:
        return ".".join([str(what), str(when), str(stat).title(), str(res)]) + ".tif"
    else:
        return ".".join([str(what), str(when), str(stat).title(), str(res), str(where)]) + ".tif"

In [13]:
def sliceRunner(top, bottom, width, outputNDV, filesDict, stats):
    assert (isinstance(bottom,int) and isinstance(top,int)
        and bottom > top)
    sliceHeight = bottom - top
    statsCalculator = TemporalAggregator_Dynamic(sliceHeight, width, outputNDV, stats)
    sliceGT = None
    sliceProj = None
    print str((top, bottom))
    isFullFile = False
    for timeKey, timeFiles in filesDict.iteritems():
        print timeKey
        for timeFile in timeFiles:
            data, thisGT, thisProj, thisNdv = ReadAOI_PixelLims(timeFile, None, (top, bottom))
            if sliceGT is None:
                # first time
                sliceGT = thisGT
                sliceProj = thisProj
                props = GetRasterProperties(timeFile)
                if sliceHeight == props["height"]:
                    isFullFile = True
            else:
                # fail in a lazy unhelpful way if the files don't all have the same geotransform and projection
                assert sliceGT == thisGT
                assert sliceProj == thisProj
            statsCalculator.addFile(data, thisNdv)
        # at end of this period get the intermediate results
        periodResults = statsCalculator.emitStep()
        if isFullFile:
            saveFolder = outDir
            where = -1
        else:
            saveFolder = tileDir
            where = top
        for stat in stats:
            SaveLZWTiff(periodResults[stat], outputNDV, sliceGT, sliceProj, saveFolder,
                       fnGetter(what, str(timeKey), stat, where))
    overallResults = statsCalculator.emitTotal()
    for stat in stats:
        if isFullFile:
            saveFolder = outDir
            where = -1
        else:
            saveFolder = tileDir
            where = top
        for stat in stats:
            SaveLZWTiff(periodResults[stat], outputNDV, sliceGT, sliceProj, saveFolder,
                       fnGetter(what, "Overall", stat, where))
            
            

In [ ]:
for t, b in slices:
    sliceRunner(t, b, fullWidth, outNdv, allFilesDict, stats)

C:\OSGEO4~1\apps\Python27\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.


(0, 8700)
2016


C:\OSGEO4~1\apps\Python27\lib\site-packages\ipykernel\__main__.py:25: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.


In [24]:
73 > np.inf

False

In [15]:
"min" in stats

True